In [218]:
import requests
import json

import re
import networkx as nx
import tqdm
import collections
import numpy as np

import matplotlib.pyplot as plt

import spacy
nlp = spacy.load('de') #load spacy model

# for WordClouds 
import matplotlib.pyplot as plt
from wordcloud import WordCloud

with open("../../data/speeches_20_withoutcomments.jsonl",'r',encoding = "utf8") as fp:
    data = list(fp)
speeches = []
for line in data:
    speeches.append(json.loads(line))

# clean party labels
for rede in speeches:
    rede['party']=rede['party'].replace(u'\xa0', u' ')
    if rede['party']=='Bündnis 90/Die Grünen':
        rede['party']='BÜNDNIS 90/DIE GRÜNEN'
    if rede['party']=='Fraktionslos':
        rede['party']='fraktionslos'

In [219]:

# last changed: 02/06/2022
def filter_for(what, search_terms, speeches):
    filtered_speeches = []
    if what == 'text':
        search_terms_low = []
        for st in search_terms:
            search_terms_low.append(st.lower())
        for speech in speeches:
            match = [st in speech[what].lower() for st in search_terms_low]
            #if all(st == True for st in match):
            if any(st in speech[what] for st in search_terms):
                #print(match)
            #if ( search_terms in speech[what] ):
                filtered_speeches.append(speech)
    else:
        for speech in speeches:
            if ( speech[what] in set(search_terms) ):
                filtered_speeches.append(speech)
        
    filtered_speeches.sort(key = lambda x:x['date'])   
    return filtered_speeches


#focal_terms = ['Digitalisierung','Zusammenhalt','Demokratie']
focal_terms = ['Digitalisierung']
#focal_terms = ['Plattform','Demokratie']
#focal_terms = ['extrem','Plattform']
#focal_terms = ['plattform','demokratie']
#focal_terms = ['Plattform','Meinung']
#focal_terms = ['Netzwerk','Meinung']
#focal_terms = ['Volk ','Volk.','Volk!','Volk?']
subset = filter_for('text', ['Klima'], speeches)
#subset = filter_for('party', ['AfD'], subset)
subset = filter_for('party', ['BÜNDNIS 90/DIE GRÜNEN'], subset)

len(subset)

133

In [221]:

def neighborhood(iterable):
    iterator = iter(iterable)
    prev_item = None
    current_item = next(iterator)  # throws StopIteration if empty.
    for next_item in iterator:
        yield (prev_item, current_item, next_item)
        prev_item = current_item
        current_item = next_item
    yield (prev_item, current_item, None)

#for prev,item,next in neighborhood(sentences):
#    print (prev, item, next )  
    
 
sentences = []
#focal_terms_sen = ['Netzwerk','sozial']
#focal_terms_sen = ['Russland']
focal_terms_sen = ['Klima']
window = 10
for rede in tqdm.tqdm(subset):
    doc = nlp(rede["text"])
    sents = doc.sents
    mysents = []
    for sx,sent in enumerate(sents):
        mysents.append(sent)
    for sx,sent in enumerate(mysents):
        #print(sent.text)
        if all(ft.lower() in sent.text.lower() for ft in focal_terms_sen):
            sentences.append(sent)
            if sx > 0 and sx < len(mysents)-1:
                sentences.append(mysents[sx-1])
                sentences.append(mysents[sx+1])
        
        
len(sentences)

100%|██████████| 133/133 [00:07<00:00, 17.20it/s]


1334

In [222]:
sentences[:10]

[Lassen Sie uns Klimaschutz und Energiewende als das sehen, was es ist: eine riesengroße Chance für Modernisierung, für Innovation und vor allen Dingen für internationale Solidarität.,
 Die wiederum hilft uns dabei, viel schneller auf 100 Prozent Erneuerbare zu kommen.,
 Denn das, was wir hier in Deutschland für Energiewende und Klimaschutz tun, hat enorme Auswirkungen auf den Rest der Welt.,
 Denn das, was wir hier in Deutschland für Energiewende und Klimaschutz tun, hat enorme Auswirkungen auf den Rest der Welt.,
 Lassen Sie uns Klimaschutz und Energiewende als das sehen, was es ist: eine riesengroße Chance für Modernisierung, für Innovation und vor allen Dingen für internationale Solidarität.,
 Die Klimakrise geht bisher vor allem zulasten der Länder im Globalen Süden, die schon jetzt die Auswirkungen am heftigsten spüren.,
 Die Klimakrise geht bisher vor allem zulasten der Länder im Globalen Süden, die schon jetzt die Auswirkungen am heftigsten spüren.,
 Denn das, was wir hier in D

In [223]:
relevantPOS = ['NOUN','ADJ','PROPN']
sentencesNN = []
words = []
for sen in sentences:
    lem = []
    for token in sen:
        if token.pos_ in relevantPOS:
            lem.append(token.lemma_)
    sentencesNN.append(lem)
    words.extend(lem)
#sentencesNN
len(words)


7332

In [226]:
zähler = collections.Counter(words).most_common()
#print(zähler)

words_clean = []
#print(focal_terms_sen)
for word in words:
    #if focal_terms_sen[0] in word or focal_terms[0] in word or 'Herr' in word or 'Dame' in word or 'Kollege' in word or 'Kollegin' in word or "/" in word:
    if 'Herr' in word or 'Dame' in word or 'Kollege' in word or 'Kollegin' in word or "/" in word:
        pass
    else:
        words_clean.append(word)
    
#print(len(words_clean))
zähler = collections.Counter(words_clean).most_common()
#print(zähler)

num_words = 1000
count = 0
new_words = []
for word in zähler:
    #print(word[0])
    if count < num_words:
        new_words.append(word[0])
        if w1 == focal_terms_sen[0]:
            wxFocus = count
    count = count + 1    

print(new_words[wxFocus])
len(new_words)        

Klimakrise


1000

In [227]:
n = len(new_words) 
A = np.ndarray(shape=(n,n), dtype=float)

for wx1,w1 in tqdm.tqdm(enumerate(new_words)):
    #print(zähler[wx1][0])

        
    for wx2,w2 in enumerate(new_words):
        if(wx2 > wx1):
            A[wx1][wx2] = 0
            for sen in sentencesNN:
                if w1 in sen and w2 in sen:
                    
                    A[wx1][wx2] = A[wx1][wx2] + 1
                    
        
            normcount = zähler[wx1][1]*zähler[wx2][1]
            #if(normcount < 2):
            #    print(normcount)
            A[wx1][wx2] = A[wx1][wx2]/normcount
            A[wx2][wx1] = A[wx1][wx2]            

#plt.imshow(A)
#plt.colorbar()
#plt.show()




0it [00:00, ?it/s]
1it [00:00,  5.06it/s]
2it [00:00,  5.20it/s]
3it [00:00,  5.38it/s]
4it [00:00,  5.49it/s]
5it [00:00,  5.70it/s]
6it [00:01,  5.69it/s]
7it [00:01,  5.64it/s]
8it [00:01,  5.77it/s]
9it [00:01,  5.82it/s]
10it [00:01,  5.94it/s]
11it [00:01,  5.97it/s]
12it [00:02,  5.96it/s]
13it [00:02,  5.94it/s]
14it [00:02,  5.96it/s]
15it [00:02,  6.06it/s]
16it [00:02,  6.09it/s]
17it [00:02,  6.01it/s]
18it [00:03,  5.70it/s]
19it [00:03,  5.62it/s]
20it [00:03,  5.30it/s]
21it [00:03,  5.17it/s]
22it [00:03,  5.04it/s]
23it [00:04,  4.91it/s]
24it [00:04,  5.39it/s]


KeyboardInterrupt: 

In [211]:
numWords = 70
final_words = []

# Numpy Array
scores = A[wxFocus]
# Getting indices of N = 3 maximum values
x = np.argsort(scores)[::-1][:numWords]
#print("Indices:",x)
# Getting N maximum values
print("Values:",scores[x])

for ele in x:
    final_words.append( new_words[ele] )

final_words

Values: [0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.05263158 0.05263158 0.03508772 0.03508772 0.03508772
 0.02631579 0.02631579 0.02631579 0.02631579 0.02631579 0.02631579
 0.02631579 0.02631579 0.02631579 0.02631579 0.02631579 0.02631579
 0.02631579 0.02631579 0.02631579 0.02631579]


['Nachhaltigkeit',
 'Biodiversität',
 'Posten',
 'Fragezeichen',
 'Fiskalausbeutung',
 'Hauptrubriken',
 'Verplanwirtschaftlichung',
 'Umerziehungsstaat',
 'Weltwirtschaftskrieg',
 'wohlwissend',
 'Tier',
 'Objekt',
 'welthistorisches',
 'Ausland',
 'Baupolitik',
 'Rebranding',
 'Nation',
 'Credo',
 'Gender',
 'Stand',
 'Kritische',
 'bestimmen',
 'Theorie',
 'Hypothese',
 'CO2-Emissionen',
 'maßgeblich',
 'Benzin',
 'eindeutig',
 'Beweis',
 'Steuerhoheit',
 'Deckmantel',
 'demokratiefeindlichen',
 'Gesundheit',
 'verlässliches',
 'Neuzuweisungen',
 'bewährt',
 'Corona',
 'Interessenpolitik',
 'globalisierter',
 'zeitgemäß',
 'vertrauensvoll',
 'Panikrhetorik',
 'Regelungswut',
 'Pump',
 'Subunternehmerin',
 'bloße',
 'Klima-und-Transformations-Sondervermögen',
 'Kardinalpunkten',
 'Verbrauch',
 'Geldbeutel',
 'Panikmodus',
 'Pflanze',
 'Rücklagen',
 'menschengemachten',
 'Kohle',
 'Wähler',
 'Zug',
 'Sprache',
 'Grundrechte',
 'drastisch',
 'stark',
 'Wende',
 'Tausende',
 'einzig',
 

In [215]:
nodes=[]
curid=1
for word in final_words:
    node = {
    'id' :  curid,
    'name' : word
    }
    nodes.append(node)
    curid=curid+1
    
graph = {
    'directed': False,
    'graph': 'word_graph',
    'links': [],
    'nodes': nodes
}

links = []
linkedwords =[]
linkedids =[]
lx = 0;
for wx1,w1 in enumerate(final_words):
    #print(wx1)
    for wx2,w2 in enumerate(final_words):
        if(wx2 > wx1) and A[wx1][wx2]>0.001:
            link_dict = {
                    'source':wx1+1,
                    'target':wx2+1,
                    'weight': A[wx1][wx2]      
                    }
            graph['links'].append(link_dict)
                      
                    #for link in links:


#graph['links']=links
#print(zähler)
print(len(graph['links']),
len(graph['nodes']))

321 70


In [125]:

#for link in graph['links']:
#    if link['weight']>1:
        #print(link)

0 100


In [216]:


graphforgephi = nx.Graph()
for node in tqdm.tqdm(graph['nodes']):
    graphforgephi.add_node(node['id'],name = node['name']);
print('nodes done')
for link in tqdm.tqdm(graph['links']):
    #weight = all((' '.join([w1['name'],w2['name']]) in linkedwords)
    #print(weight)         
    graphforgephi.add_edge(link['source'],link['target'],weight=link['weight'])
print('links done')   
nx.write_gexf(graphforgephi, "graphforgephi.gexf")
print('save done')

100%|██████████| 321/321 [00:00<00:00, 291296.32it/s]

nodes done
links done
save done


In [ ]:
            for sen in sentencesNN:
                if w1['name'] in sen and w2['name'] in sen:
                    weight = len([ele for ele in linkedwords if ele == (' '.join([w1['name'],w2['name']]))])
                    #    links[]
                    #else:
                    #print(weight)
                    linkedwords.append(' '.join([w1['name'],w2['name']]))
                    #linkedids[ w1['id'] ] , w2['id'] )
                    link_dict = {
                    'source':w1['id'],
                    'target':w2['id'],
                    'sourceWD':w1['name'],
                    'targetWD':w2['name'],
                    'weight': weight+1      
                    }
                    #print(link_dict)
                    links.append(link_dict)
                    graph['links'].append(link_dict)